# Historical Tonnage List API Examples

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
And put your API key in a `SIGNAL_OCEAN_API_KEY` environment variable.

## Example 1 - Plotting a supply trend
The data frame format makes it very easy to generate a supply trend plot.

We'll generate a supply trend from 10 days, but we'll also filter the vessel list by looking for vessels that:
- are pushed, poss_pushed,
- have a market deployment type of "Relet" or "Spot",
- their commercial status is available, cancelled or failed,
- are crude oil tankers (their vessel subclass is "Dirty"),
- their AIS information is no older than 5 days.

Filtering can be achieved by creating an instance of a `VesselFilter` and passing it to the `get_historical_tonnage_list` method. A `VesselFilter` meeting the above criteria will look as follows:

In [3]:
from datetime import date, timedelta,time
from signal_ocean import PortAPI, VesselClassAPI,PortFilter,VesselClassFilter,Connection
from signal_ocean.historical_tonnage_list import VesselFilter, PushType, MarketDeployment, CommercialStatus, VesselSubclass
from signal_ocean.historical_tonnage_list import HistoricalTonnageListAPI

connection = Connection('262c367b7782482e99f09793570eee9a')

port_api = PortAPI(connection)
vessel_class_api = VesselClassAPI(connection)
htl_api = HistoricalTonnageListAPI(connection)
vessel_class = vessel_class_api.get_vessel_classes(VesselClassFilter(name_like='suezmax'))[0]
port = port_api.get_ports(PortFilter(name_like='escravos'))[0]
laycanEndInDays = 20
start_date = date.today() - timedelta(days=10)
end_date = date.today()

vessel_filter = VesselFilter(
    push_types=[PushType.PUSHED, PushType.PUSHED_POSS],
    market_deployments=[MarketDeployment.RELET, MarketDeployment.SPOT],
    commercial_statuses=[CommercialStatus.AVAILABLE, CommercialStatus.CANCELLED, CommercialStatus.FAILED],
    vessel_subclass=VesselSubclass.DIRTY,
    latest_ais_since=5
)


htl_for_supply_trend = htl_api.get_historical_tonnage_list(
    port,
    vessel_class,
    laycanEndInDays,
    start_date,
    end_date,
    vessel_filter=vessel_filter,
    time=time(hour=6)
)

supply_trend_data_frame = htl_for_supply_trend.to_data_frame()

supply_trend_data_frame

name vessel_class ice_class  year_built  \
date       imo                                                         
2020-08-06 9819868       Harmonic      Suezmax       NaN        2019   
           9408695   Front Balder      Suezmax       NaN        2009   
           9600865      Nantucket      Suezmax       NaN        2014   
           9233765  Nordic Cosmos      Suezmax       NaN        2003   
           9232931      SKS Sinni      Suezmax       NaN        2003   
...                           ...          ...       ...         ...   
2020-07-27 9302994         Selena      Suezmax       NaN        2007   
           9585900   Ast Sunshine      Suezmax       NaN        2013   
           9318539          Donat      Suezmax       NaN        2007   
           9411226     Ce-Bermuda      Suezmax       NaN        2009   
           9299678       Euronike      Suezmax       NaN        2005   

                    deadweight  length_overall  breadth_extreme  \
date       imo                                                    
2020-08-06 9819868      159204          274.30               48   
           9408695      156436          274.50               48   
           9600865      156902          274.50               48   
           9233765      159999          274.20               48   
           9232931      159385          274.20               48   
...                        ...             ...              ...   
2020-07-27 9302994      150296          274.20               48   
           9585900      159039          274.00               48   
           9318539      166188          281.20               48   
           9411226      158143          274.34               48   
           9299678      164565          274.00               50   

                   market_deployment_point_in_time push_type_point_in_time  \
date       imo                                                               
2020-08-06 9819868                            Spot             Pushed POSS   
           9408695                            Spot                  Pushed   
           9600865                            Spot                  Pushed   
           9233765                            Spot                  Pushed   
           9232931                            Spot                  Pushed   
...                                            ...                     ...   
2020-07-27 9302994                            Spot                  Pushed   
           9585900                            Spot             Pushed POSS   
           9318539                           Relet                  Pushed   
           9411226                            Spot             Pushed POSS   
           9299678                            Spot                  Pushed   

                   open_port_point_in_time  ...  latest_ais_point_in_time  \
date       imo                              ...                             
2020-08-06 9819868                  Espoir  ... 2020-08-06 00:00:00+00:00   
           9408695                 Kizomba  ... 2020-08-06 00:00:00+00:00   
           9600865                 Kizomba  ... 2020-08-06 00:00:00+00:00   
           9233765                    Lome  ... 2020-08-06 00:00:00+00:00   
           9232931                    Lome  ... 2020-08-06 00:00:00+00:00   
...                                    ...  ...                       ...   
2020-07-27 9302994                  Donges  ... 2020-07-27 00:00:00+00:00   
           9585900                 Milazzo  ... 2020-07-27 00:00:00+00:00   
           9318539                  Aliaga  ... 2020-07-27 00:00:00+00:00   
           9411226                     Fos  ... 2020-07-27 00:00:00+00:00   
           9299678                     Fos  ... 2020-07-27 00:00:00+00:00   

                   subclass open_prediction_accuracy_point_in_time  \
date       imo                                                       
2020-08-06 9819868    Dirty                            Narrow A

Now, we can generate the plot:

In [ ]:
from signal_ocean.historical_tonnage_list import IndexLevel

supply_trend = supply_trend_data_frame.groupby(IndexLevel.DATE, sort=True).size()
plot = supply_trend.plot()
plot.set_ylabel('Vessel count')

plot

## Example 2 - Generating an Excel sheet
The data frame can be easily saved as an Excel file by using Pandas's built-in `to_excel()` function.

Before we do that, we need to remove all the time zone information from all the timestamps in the data frame. This is because Excel does not support storing time zone information along with timestamps. However, Signal Ocean's SDK always provides time zone information to make all timestamp-based computation unambiguous.

In [ ]:
from signal_ocean.historical_tonnage_list import Column

without_time_zones = (
    supply_trend_data_frame
        .reset_index()
        .astype({ Column.OPEN_DATE: 'datetime64[ns]', Column.ETA: 'datetime64[ns]', Column.LATEST_AIS: 'datetime64[ns]'})
        .set_index([IndexLevel.DATE, IndexLevel.IMO])
 )

Now, we can generate the Excel file:

In [ ]:
without_time_zones.to_excel('simpleDemo.xlsx')